### **Búsqueda y Minería de Información 2021-22**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Grado en Ingeniería Informática, 4º curso
# **Motores de búsqueda e indexación**

Fechas:

* Comienzo: lunes 21 / martes 22 de febrero
* Entrega: lunes 28 / martes 29 de marzo (14:00)

# Introducción

## Objetivos

Los objetivos de esta práctica son:

* La implementación eficiente de funciones de ránking, particularizada en el modelo vectorial.
*	La implementación de índices eficientes para motores de búsqueda. 
*	La implementación de un método de búsqueda proximal.
*	La dotación de estructuras de índice posicional que soporten la búsqueda proximal.
*	La implementación del algoritmo PageRank.

Se desarrollarán implementaciones de índices utilizando un diccionario y listas de postings. Y se implementará el modelo vectorial utilizando estas estructuras más eficientes para la ejecución de consultas.

Los ejercicios básicos consistirán en la implementación de algoritmos y técnicas estudiados en las clases de teoría, con algunas propuestas de extensión opcionales. Se podrá comparar el rendimiento de las diferentes versiones de índices y buscadores, contrastando la coherencia con los planteamientos estudiados a nivel teórico.

Mediante el nivel de abstracción seguido, se conseguirán versiones intercambiables de índices y buscadores. El único buscador que no será intercambiable es el de Whoosh, que sólo funcionará con sus propios índices.

## Material proporcionado

Se proporcionan (bien en el curso de Moodle o dentro de este documento):

*	Varias clases e interfaces Python a lo largo de este *notebook*, con las que el estudiante integrará las suyas propias. 
Las clases parten del código de la práctica anterior.
Igual que en la práctica 1, la función **main** implementa un programa que deberá funcionar con las clases a implementar por el estudiante.
*	Las colecciones de prueba de la práctica 1: <ins>toys.zip</ins> (que se descomprime en dos carpetas toy1 y toy2), <ins>docs1k.zip</ins> con 1.000 documentos HTML y un pequeño fichero <ins>urls.txt</ins>. 
*	Una colección más grande: <ins>docs10k.zip</ins> con 10.000 documentos HTML.
*	Varios grafos para probar PageRank: <ins>graphs.zip</ins>.
*	Un documento de texto <ins>output.txt</ins> con la salida estándar que deberá producir la ejecución de la función main.

In [ ]:
import os, os.path
import re
import math
import pickle
import zipfile
from abc import ABC, abstractmethod
from urllib.request import urlopen
from bs4 import BeautifulSoup

class Config(object):
  # variables de clase
  NORMS_FILE = "docnorms.dat"
  PATHS_FILE = "docpaths.dat"
  INDEX_FILE = "serialindex.dat"
  DICTIONARY_FILE = "dictionary.dat"
  POSTINGS_FILE = "postings.dat"

class BasicParser:
    @staticmethod
    def parse(text):
        return re.findall(r"[^\W\d_]+|\d+", text.lower())

def tf(freq):
    return 1 + math.log2(freq) if freq > 0 else 0

def idf(df, n):
    return math.log2((n + 1) / (df + 0.5))

"""
    This is an abstract class for the search engines
"""
class Searcher(ABC):
    def __init__(self, index, parser=BasicParser()):
        self.index = index
        self.parser = parser
    @abstractmethod
    def search(self, query, cutoff):
        """ Returns a list of documents encapsulated in a SearchRanking class """

class Index:
    def __init__(self, dir=None):
        self.docmap = []
        self.modulemap = {}
        if dir: self.open(dir)

    def add_doc(self, path):
        self.docmap.append(path)  # Assumed to come in order

    def doc_path(self, docid):
        return self.docmap[docid]

    def doc_module(self, docid):
        if docid in self.modulemap:
            return self.modulemap[docid]
        return None

    def ndocs(self):
        return len(self.docmap)

    def doc_freq(self, term):
        return len(self.postings(term))

    def term_freq(self, term, docID):
        post = self.postings(term)
        if post is None: return 0
        for posting in post:
            if posting[0] == docID:
                return posting[1]
        return 0

    def total_freq(self, term):
        freq = 0
        for posting in self.postings(term):
            freq += posting[1]
        return freq

    def postings(self, term):
        # used in more efficient implementations
        return list()

    def positional_postings(self, term):
        # used in positional implementations
        return list()

    def all_terms(self):
        return list()

    def save(self, dir_path):
        if not self.modulemap: self.compute_modules()
        p = os.path.join(dir_path, Config.NORMS_FILE)
        with open(p, 'wb') as f:
            pickle.dump(self.modulemap, f)        

    def open(self, dir_path):
        try:
            p = os.path.join(dir_path, Config.NORMS_FILE)
            with open(p, 'rb') as f:
                self.modulemap = pickle.load(f)
        except OSError:
            # the file may not exist the first time
            pass

    def compute_modules(self):
        for term in self.all_terms():
            idf_score = idf(self.doc_freq(term), self.ndocs())
            post = self.postings(term)
            if post is None: continue
            for docid, freq in post:
                if docid not in self.modulemap: self.modulemap[docid] = 0
                self.modulemap[docid] += math.pow(tf(freq) * idf_score, 2)
        for docid in range(self.ndocs()):
            self.modulemap[docid] = math.sqrt(self.modulemap[docid]) if docid in self.modulemap else 0

class Builder:

    def __init__(self, dir_path, parser=BasicParser()):
        if os.path.exists(dir_path): shutil.rmtree(dir_path)
        os.makedirs(dir_path)
        self.parser = parser

    def build(self, path):
        if zipfile.is_zipfile(path):
            self.index_zip(path)
        elif os.path.isdir(path):
            self.index_dir(path)
        else:
            self.index_url_file(path)

    def index_zip(self, filename):
        file = zipfile.ZipFile(filename, mode='r', compression=zipfile.ZIP_DEFLATED)
        for name in sorted(file.namelist()):
            with file.open(name, "r", force_zip64=True) as f:
                self.index_document(name, BeautifulSoup(f.read().decode("utf-8"), "html.parser").text)
        file.close()

    def index_dir(self, dir_path):
        for subdir, dirs, files in os.walk(dir_path):
            for file in sorted(files):
                path = os.path.join(dir_path, file)
                with open(path, "r") as f:
                    self.index_document(path, f.read())

    def index_url_file(self, file):
        with open(file, "r") as f:
            self.index_urls(line.rstrip('\n') for line in f)

    def index_urls(self, urls):
        for url in urls:
            self.index_document(url, BeautifulSoup(urlopen(url).read().decode("utf-8"), "html.parser").text)
    
    def index_document(self, path, text):
        pass
    
    def commit(self):
        pass

In [ ]:
# from previous lab
class SlowVSMSearcher(Searcher):
    
    def __init__(self, index, parser=BasicParser()):
        super().__init__(index, parser)

    def search(self, query, cutoff):
        qterms = self.parser.parse(query)
        ranking = SearchRanking(cutoff)
        for docid in range(self.index.ndocs()):
            score = self.score(docid, qterms)
            if score:
                ranking.push(self.index.doc_path(docid), score)
        return ranking

    def score(self, docid, qterms):
        prod = 0
        for term in qterms:
            prod += tf(self.index.term_freq(term, docid)) \
                    * idf(self.index.doc_freq(term), self.index.ndocs())
        mod = self.index.doc_module(docid)
        if mod:
            return prod / mod
        return 0

In [ ]:
try:
  import whoosh
except ModuleNotFoundError:
  !pip install whoosh
  import whoosh
from whoosh.fields import Schema, TEXT, ID
from whoosh.formats import Format
from whoosh.qparser import QueryParser

# A schema in Whoosh is the set of possible fields in a document in
# the search space. We just define a simple 'Document' schema, with
# a path (a URL or local pathname) and a content.
SimpleDocument = Schema(
        path=ID(stored=True),
        content=TEXT(phrase=False))
ForwardDocument = Schema(
        path=ID(stored=True),
        content=TEXT(phrase=False,vector=Format))
PositionalDocument = Schema(
        path=ID(stored=True),
        content=TEXT(phrase=True))

class WhooshBuilder(Builder):
    def __init__(self, dir, schema=SimpleDocument):
        super().__init__(dir)
        self.whoosh_writer = whoosh.index.create_in(dir, schema).writer(procs=1, limitmb=16384, multisegment=True)
        self.dir = dir

    def index_document(self, p, text):
        self.whoosh_writer.add_document(path=p, content=text)

    def commit(self):
        self.whoosh_writer.commit()
        index = WhooshIndex(self.dir)
        index.save(self.dir)

class WhooshForwardBuilder(WhooshBuilder):
    def __init__(self, dir_path):
        super().__init__(dir_path, ForwardDocument)

    def commit(self):
        self.whoosh_writer.commit()
        index = WhooshForwardIndex(self.dir)
        index.save(self.dir)

class WhooshPositionalBuilder(WhooshBuilder):
    def __init__(self, dir_path):
        super().__init__(dir_path, PositionalDocument)

    def commit(self):
        self.whoosh_writer.commit()
        index = WhooshPositionalIndex(self.dir)
        index.save(self.dir)

class WhooshIndex(Index):
    def __init__(self, dir):
        super().__init__(dir)
        self.whoosh_reader = whoosh.index.open_dir(dir).reader()    

    def total_freq(self, term):
        return self.whoosh_reader.frequency("content", term)

    def doc_freq(self, term):
        return self.whoosh_reader.doc_frequency("content", term)

    def doc_path(self, docid):
        return self.whoosh_reader.stored_fields(docid)['path']

    def ndocs(self):
        return self.whoosh_reader.doc_count()

    def all_terms(self):
        return list(self.whoosh_reader.field_terms("content"))

    def postings(self, term):
        return self.whoosh_reader.postings("content", term).items_as("frequency") \
            if self.doc_freq(term) > 0 else []

class WhooshForwardIndex(WhooshIndex):
    def term_freq(self, term, docID) -> int:
        if self.whoosh_reader.has_vector(docID, "content"):
            v = self.whoosh_reader.vector(docID, "content")
            v.skip_to(term)
            if v.id() == term:
                return v.value_as("frequency")
        return 0

class WhooshPositionalIndex(WhooshIndex):
    def positional_postings(self, term):
        return self.whoosh_reader.postings("content", term).items_as("positions") \
            if self.doc_freq(term) > 0 else []

class WhooshSearcher(Searcher):
    def __init__(self, dir):
        self.whoosh_index = whoosh.index.open_dir(dir)
        self.whoosh_searcher = self.whoosh_index.searcher()
        self.qparser = QueryParser("content", schema=self.whoosh_index.schema)

    def search(self, query, cutoff):
        return map(lambda scoredoc: (self.doc_path(scoredoc[0]), scoredoc[1]),
                   self.whoosh_searcher.search(self.qparser.parse(query), limit=cutoff).items())

    def doc_path(self, docid):
        return self.whoosh_index.reader().stored_fields(docid)['path']

## Calificación

Esta práctica se calificará con una puntuación de 0 a 10 atendiendo a las puntuaciones individuales de ejercicios y apartados dadas en el enunciado. No obstante, aquellos ejercicios marcados con un asterisco (*) tienen una complejidad un poco superior a los demás (que suman 7.5 puntos), y permiten, si se realizan todos, una nota superior a 10. 

El peso de la nota de esta práctica en la calificación final de prácticas es del **40%**.

La calificación se basará en a) el **número** de ejercicios realizados y b) la **calidad** de los mismos. La calidad se valorará por los **resultados** conseguidos (economía de consumo de RAM, disco y tiempo; tamaño de las colecciones que se consigan indexar) pero también del **mérito** en términos del interés de las técnicas aplicadas y la buena programación.

La puntuación que se indica en cada apartado es orientativa, en principio se aplicará tal cual se refleja pero podrá matizarse por criterios de buen sentido si se da el caso.

Para dar por válida la realización de un ejercicio, el código deberá funcionar (a la primera) integrado con las clases que se facilitan. El profesor comprobará este aspecto añadiendo los módulos entregados por el estudiante a los módulos facilitados en la práctica, ejecutando la función *main* así como otros main de prueba adicionales.

## Entrega

La entrega consistirá en un único fichero tipo *notebook* donde se incluirán todas las **implementaciones** solicitadas en cada ejercicio, así como una explicación de cada uno a modo de **memoria**.

## Indicaciones

Se sugiere trabajar en la práctica de manera incremental, asegurando la implementación de soluciones sencillas y mejorándolas de forma modular (la propia estructura de ejercicios plantea ya esta forma de trabajar).

Se podrán definir clases o módulos adicionales a las que se indican en el enunciado, por ejemplo, para reutilizar código. Y el estudiante podrá utilizar o no el software que se le proporciona, con la siguiente limitación: la función **main** deberá ejecutar correctamente <ins>sin ninguna modificación</ins> (más allá de comentar aquellos ejercicios que no se hayan realizado).

Asimismo, se recomienda indexar sin ningún tipo de stopwords ni stemming, para poder hacer pruebas más fácilmente con ejemplos “de juguete”.

# Ejercicio 1: Implementación de un modelo vectorial eficiente

Se mejorará la implementación de la práctica anterior aplicando algoritmos estudiados en las clases de teoría. En particular, se utilizarán listas de postings en lugar de un índice forward.

La reimplementación seguirá haciendo uso de la clase abstracta Index, y se podrá probar con cualquier implementación de esta clase (tanto la implementación de índice sobre Whoosh como las propias). 

## Ejercicio 1.1: Método orientado a términos (3pt)

Escribir una clase TermBasedVSMSearcher que implemente el modelo vectorial coseno por el método orientado a términos.

In [ ]:
from typing import Dict, List
from collections import OrderedDict
import heapq

class TermBasedVSMSearcher(Searcher):
    # Your new code here (exercise 1.1) #
    def __init__(self, index, parser=BasicParser()):
        super().__init__(index, parser)

    def search(self, query, cutoff):
        q_terms = self.parser.parse(query)
        ranking = SearchRanking(cutoff)
        posting_vals: Dict = {}

        for term in q_terms:
            for docid, _ in self.index.postings(term):
                if docid not in posting_vals.keys():
                    posting_vals[docid] = []
                posting_vals[docid].append(term)

        for docid in posting_vals.keys():
            score = self.score(docid, posting_vals[docid])
            if score:
                ranking.push(self.index.doc_path(docid), score)

        return ranking

    def score(self, docid, q_terms):
        prod = 0
        for term in q_terms:
            prod += tf(self.index.term_freq(term, docid)) * idf(self.index.doc_freq(term), self.index.ndocs())

        mod = self.index.doc_module(docid)
        if mod:
            return prod / mod
         
        return 0

### Explicación/documentación

Al ser una búsqueda orientada a términos, iteraremos primero sobre los términos, de tal forma que para cada término que buscamos, recorreremos todos los documentos, asociando el docid de cada documento.

Para calcular el score, obtendremos resultados parciales del producto tf*idf, esto quiere decir que cada término tendrá varios resultados, teniendo que sumarlos sucesivamente para calcular el score final (ya que hemos iterado sobre términos primero, habrá que hacer lo mismo al calcular el score). Finalmente se calcula el módulo.

## Ejercicio 1.2: Método orientado a documentos* (1pt)

Implementar el método orientado a documentos (con heap de postings) en una clase DocBasedVSMSearcher.

In [ ]:
class DocBasedVSMSearcher(Searcher):
    def __init__(self, index, parser=BasicParser()):
        super().__init__(index, parser)

    def search(self, query, cutoff):
        q_terms = self.parser.parse(query)
        posting_list: List = []
        ndocs = self.index.ndocs()
        rank = SearchRanking(cutoff)
        score = [0] * ndocs
        index = [0] * len(q_terms)

        # Recuperar lista de postings # 
        for q in q_terms:
            posting_list += [[p for p in self.index.postings(q)]]
        
        heap = []
        heapq.heapify(heap)
        #Tras crear un min heap, iteramos consecuencialmente en orden de docID 
        for q in range(len(q_terms)):
            heapq.heappush(heap, (posting_list[q][index[q]], q))
            index[q] += 1 
        
        docActual = heapq.nsmallest(1, heap)[0][0][0]
        try:
            # Obtenemos elemento, docID, frecuencia y el score (que sumaremos en el caso de haber más en el mismo docID)
            while(1):
                element = heapq.heappop(heap)
                docID = element[0][0]
                freq = element[0][1]
                q = element[1]
                partial_score = tf(freq) * idf(self.index.doc_freq(q_terms[q]), ndocs)

                # Cuando cambiamos de docID, sumaremos todos los scores parciales #
                if docID != docActual:
                    score[docActual] /= self.index.doc_module(docActual)
                    # Tras hacer la operación del módulo, si el heap está incompleto se añade directamente el docID y su score
                    if len(rank.ranking) < rank.cutoff:
                        rank.push(self.index.doc_path(
                            docActual), rank[docActual])
                    # Si el heap está completo, entonces hay que ver si el score total es mayor que el menor del heap (ya que es un min heap)
                    elif score > heapq.nsmallest(1, rank.ranking)[0][0]:
                        rank.push(self.index.doc_path(
                            docActual), rank[docActual])

                    docActual = docID
                rank[docActual] += partial_score

                # Si no se llega al número de índices de la lista de posting, entonces se añade al heap de Ranking
                if index[q] < len(posting_list[q]):
                    heapq.heappush(heap, (posting_list[q][index[q]], q))
                    index[q] += 1

        # El heap está vacío
        except Exception:  
            score[docActual] /= self.index.doc_module(docActual)
            if len(rank.ranking) < rank.cutoff:
                rank.push(self.index.doc_path(docActual), score[docActual])
            elif score > heapq.nsmallest(1, rank.ranking)[0][0]:
                rank.push(self.index.doc_path(docActual), score[docActual])

        return rank

### Explicación/documentación

Este método está orientado a documentos, y por lo tanto la lista de postings estará ordenado por docIDs. Construiremos un min heap (del tamaño de la query) y tras iterar consecuencialmente en función del docID, llenaremos el heap.

A continuación, obtenemos el docID más pequeño y cogemos el elemento, su frecuencia y el tf*idf como score parcial (en función del número de veces que esté ese docID tendremos más scores parciales). Una vez que hemos vaciado el heap de un docID, dividiremos el sumatorio de los scores parciales entre el módulo y lo meteremos al min heap si ese score final es mayor que el menor score del heap.

Finalmente, dará una excepción en el caso de que el heap esté vacío y se devuelve el heap ranking.

## Ejercicio 1.3: Heap de ránking (0.5pt)

Reimplementar la clase entregada SearchRanking para utilizar un heap de ránking (se recomienda usar el módulo [heapq](https://docs.python.org/3/library/heapq.html)). Nótese que esta opción se aprovecha mejor con la implementación orientada a documentos, aunque es compatible con la orientada a términos.

In [ ]:
class SearchRanking:
    # TODO: to be implemented as heap (exercise 1.3) #
    def __init__(self, cutoff):
        self.ranking: List = [] # implementation as list, not as heap! TO BE MODIFIED
        heapq.heapify(self.ranking)
        self.cutoff = cutoff

    def push(self, docid, score):
        #self.ranking.append((docid, score))
        heapq.heappush(self.ranking, (score,docid))

    def __iter__(self):
        return iter([tuple(reversed(e)) for e in heapq.nlargest(min(len(self.ranking), self.cutoff), self.ranking)])
        min_l = min(len(self.ranking), self.cutoff)
        ## sort ranking
        self.ranking.sort(key=lambda tup: tup[1], reverse=True)
        return iter(self.ranking[0:min_l])

### Explicación/documentación

La clase SearchRanking tiene que ser reimplementada, dejando de ser una lista para ser un heap. Esto lo hacemos gracias a la librería heapq con la función "heapify", que convierte una lista en un MinHeap. El cutoff se mantiene solo que ahora se hará en el heap en vez de en la lista.

Para pushear, simplemente utilizamos la función "heappush" dada por heapq, y pusheamos una tupla que guarda el score y el id del documento. Que push sea de esta forma (score,docid) se debe a que luego, para iterar, lo haremos en función del score, ya que queremos que esté ordenado por la puntuación que tiene antes que por el docid (ya que además un término puede aparecer en varios documentos).

Finalmente se hace el ranking teniendo en cuenta el cutoff previamente especificado.

# Ejercicio 2: Índice en RAM (3pt)

Implementar un índice propio que pueda hacer las mismas funciones que la implementación basada en Whoosh definida en la práctica 1. Como primera fase más sencilla, los índices se crearán completamente en RAM. Se guardarán a disco y leerán de disco en modo serializado (ver módulo [pickle](https://docs.python.org/3/library/pickle.html)).

Para guardar el índice se utilizarán los nombres de fichero definidos por las variables estáticas de la clase Config. 

Antes de guardar el índice, se borrarán todos los ficheros que pueda haber creados en el directorio del índice. Asimismo, el directorio se creará si no estuviera creado, de forma que no haga falta crearlo a mano. Este detalle se hará igual en los siguientes ejercicios.

## Ejercicio 2.1: Estructura de índice

Implementar la clase RAMIndex como subclase de Index con las estructuras necesarias: diccionario, listas de postings, más la información que se necesite. 

Para este ejercicio en las listas de postings sólo será necesario guardar los docIDs y las frecuencias; no es necesario almacenar las posiciones de los términos.

In [ ]:
class RAMIndex(Index):
    def __init__(self, dir_path=None):
        self.terms_dict = {}
        super().__init__(dir_path)

    def postings(self, term):
        return self.terms_dict[term]

    def all_terms(self):
        return self.terms_dict.keys()

    def save(self, dir_path):
        f = open(os.path.join(dir_path, Config.PATHS_FILE), 'wb')
        pickle.dump(self.docmap, f)
        f.close()
        f = open(os.path.join(dir_path, Config.DICTIONARY_FILE), 'wb')
        self.terms_dict = OrderedDict(sorted(self.terms_dict.items()))
        pickle.dump(self.terms_dict, f)
        f.close()
        super().save(dir_path)

    def open(self, dir_path):
        super().open(dir_path)
        try:
            f = open(os.path.join(dir_path, Config.PATHS_FILE), 'rb')
            self.docmap = pickle.load(f)
            f.close()
            f = open(os.path.join(dir_path, Config.DICTIONARY_FILE), 'rb')
            self.terms_dict = pickle.load(f)
            f.close()
        except OSError:
            pass

### Explicación/documentación

Al guardar los términos y los postings en memoria, solo se inicializa un diccionario (que al principio está vacío). Para obtener los postings de un término solo hay que buscar en el diccionario el término que nos interesa y, para obtener todos los términos del índice se devuelven las claves del diccionario (que son, de hecho, los términos).

Para guardar, mediante la librería pickle, podemos meter todo el diccionario en un fichero (DICTIONARY_FILE) con la función "dump", por lo tanto todos los datos del diccionario están condensados en el fichero de esta forma es más fácil cargar los datos con la función "load", ya que simplemente obtenemos todos los datos del diccionario que hemos guardado en el fichero anterior.

## Ejercicio 2.2 Construcción del índice

Implementar la clase RAMIndexBuilder como subclase de Builder, que cree todo el índice en RAM a partir de una colección de documentos.

In [ ]:
class RAMIndexBuilder(Builder):
    # Your new code here (exercise 2.2) #
    def __init__(self, dir_path, parser=BasicParser()):
        super().__init__(dir_path, parser)
        self.path = dir_path
        self.index = RAMIndex(dir_path)

    def index_document(self, path, text):
        doc_id = self.index.ndocs()

        self.index.add_doc(path)

        text = BasicParser.parse(text)
        term_freq: Dict = {}

        for term in text:
            if self.index.terms_dict.get(term) is None:
                term_freq[term] = 0
                self.index.terms_dict[term] = []

            elif term_freq.get(term) is None:
                term_freq[term] = 0

            term_freq[term] += 1

        for term in term_freq.keys():
            self.index.terms_dict[term].append(
                [doc_id, term_freq[term]])

    def commit(self):
        self.index.save(self.path)  
        f = open(self.path + Config.INDEX_FILE, "wb")
        pickle.dump(self.index, f)
        f.close()
        return

### Explicación/documentación

Para indexar un término iteraremos sobre el texto de los documentos que hemos recogido previamente, de tal forma que si es la primera vez que nos encontramos un términos lo añadimos a un diccionario donde guardaremos tanto término como frecuencia. Inicializamos el valor de la frecuencia a 0, de tal forma que cada vez que aparezca el término se sumará uno a la frecuencia.

Después de comprobar si el término está ya o no en el diccionario y sumar la frecuencia en el caso de que ya estuviera, añadimos al término el id del documento donde aparece y la frecunecia del término, dando lugar a la lista de postings.

Al hacer commit, el índice se guarda y se vuelca la información referente al mismo en el fichero INDEX_FILE.

# Ejercicio 3: Índice en disco* (1pt)

Reimplementar los índices definiendo las clases DiskIndex y DiskIndexBuilder de forma que:

*	El índice se siga creando entero en RAM (por ejemplo, usando estructuras similares a las del ejercicio 2).
*	Pero el índice se guarde en disco dato a dato (docIDs, frecuencias, etc.).
*	Al cargar el índice, sólo el diccionario se lee a RAM, y se accede a las listas de postings en disco cuando son necesarias (p.e. en tiempo de consulta).

Se sugiere guardar el diccionario en un fichero y las listas de postings en otro, utilizando los nombres de fichero definidos como variables estáticas en la clase Config.

Observación: se sugiere inicialmente guardar en disco las estructuras de índice en modo texto para poder depurar los programas. Una vez asegurada la corrección de los programas, puede ser más fácil pasar a modo binario o serializable (usando el módulo pickle como en ejercicios previos).

In [ ]:
class DiskIndex(Index):
    # Your new code here (exercise 3*) #
    def __init__(self, dir_path=None):
        self.postings_dict: Dict = {}
        self.terms_dict: Dict = {}
        self.postings_path = os.path.join(dir_path, Config.POSTINGS_FILE)
        self.count = 0
        super().__init__(dir_path)
        
    def postings(self, term):
        postings_ret: List = []
        f = open(self.postings_path, 'r')
        f.seek(self.terms_dict[term])

        doc_amount, raw_postings = f.readline().strip().split("-")
        raw_postings = raw_postings.split(" ")

        for i in range(0, int(doc_amount) * 2, 2):
            postings_ret.append([int(raw_postings[i]), int(raw_postings[i+1])])

        f.close()
        return postings_ret

    def all_terms(self):
        return self.terms_dict.keys()

    def doc_freq(self, term):
        f = open(self.postings_path, 'r')
        f.seek(self.terms_dict[term])
        result = f.readline().split("-")[0]
        f.close()
        return int(result)

    def save(self, dir_path):
        f = open(os.path.join(dir_path, Config.PATHS_FILE), 'wb')
        pickle.dump(self.docmap, f)
        f.close()
        
        f = open(os.path.join(dir_path, Config.POSTINGS_FILE), 'w')
        for term, postings in self.postings_dict.items():
                self.terms_dict[term] = f.tell()
                n_postings = len(postings)
                f.write(str(n_postings) + "-")
                for p in postings:
                    f.write(str(p[0]) + " ")
                    f.write(str(p[1]) + " ")
                f.write("\n")
        f.close()
        
        f = open(os.path.join(dir_path, Config.DICTIONARY_FILE), 'wb')
        self.terms_dict = OrderedDict(sorted(self.terms_dict.items()))
        pickle.dump(self.terms_dict, f)
        f.close()

        super().save(dir_path)


    def open(self, dir_path):
        super().open(dir_path)
        self.postings_dict = {}
        try:
            f = open(os.path.join(dir_path, Config.PATHS_FILE), 'rb')
            self.docmap = pickle.load(f)
            f.close()
            f = open(os.path.join(dir_path, Config.DICTIONARY_FILE), 'rb')
            self.terms_dict = pickle.load(f)
            f.close()
        except OSError:
            pass


class DiskIndexBuilder(Builder):
    # Your new code here (exercise 3*) #
    def __init__(self, dir_path, parser=BasicParser()):
        super().__init__(dir_path, parser)
        self.path = dir_path
        self.index = DiskIndex(dir_path)

    def index_document(self, path, text):
        doc_id = self.index.ndocs()

        self.index.add_doc(path)

        text = BasicParser.parse(text)
        term_freq: Dict = {}

        for term in text:
            if self.index.postings_dict.get(term) is None:
                term_freq[term] = 0
                self.index.postings_dict[term] = []

            elif term_freq.get(term) is None:
                term_freq[term] = 0

            term_freq[term] += 1

        for term in term_freq.keys():
            self.index.postings_dict[term].append(
                [doc_id, term_freq[term]])

    def commit(self):
        self.index.save(self.path)  
        f = open(self.path + Config.INDEX_FILE, "wb")
        pickle.dump(self.index, f)
        f.close()


### Explicación/documentación

3.1 - DiskIndex

Empezaremos hablando del planteamiento seguido para el indice, en este caso tenemos mas estructuras que en el RamIndex aunque compartimos el diccionario de terminos. Como novedad vemos la agregacion de un diccionario de postings, otro de terminos y sus offsets para poder acceder a la memoria, otro diccionario con los paths.

En este caso podemos ver una funcion de postings algo mas elaborada, creadno una estructura simple a la hora de trabajar y procesar, guardamos la posicion que indique el offset para dicho termino, asi podemos leer directamente la fila del documento cargando los postings.

Otra funcion modificada en este caso es la funcion doc_freq con respecto a la clase base de Index. Para no tener que leer la lista de postings cada vez, a la hora de guardar la lista de postings en la funcion save hemos agregado el valor, necesitando asi solo leer el valor del fichero.

Y hablando del save, tantno la funcion save como open hacen uso de pickle para el formato del fichero.

3.2 - DiskIndexBuilder

Al contrario que el DiskIndex que tiene deiferencias a la hora de controlar los datos, el builder es igual al de RAMIndex.

# Ejercicio 4: Motor de búsqueda proximal* (1pt)

Implementar un método de búsqueda proximal en una clase ProximitySearcher, utilizando las interfaces de índices posicionales. Igual que en los ejercicios anteriores, se sugiere definir esta clase como subclase (directa o indirecta) de Searcher. Para empezar a probar este buscador, se proporciona una implementación de indexación posicional basada en Whoosh (WhooshPositionalIndex).

In [ ]:
class ProximitySearcher(Searcher):
    # Your new code here (exercise 4*) #
    def __init__(self, index, parser=BasicParser()):
        super().__init__(index, parser)
    
    def search(self, query, cutoff):
        q_terms = self.parser.parse(query)
        rank = SearchRanking(cutoff)
        new_doc_ids = set()
        final_positions = {}

        for doc_id, _ in self.index.postings(q_terms[0]):
            new_doc_ids.add(doc_id)

        if len(q_terms) > 1:
            for term in q_terms[1:]:
                old_doc_ids = set()
                for doc_id, _ in self.index.postings(term):
                    old_doc_ids.add(doc_id)
                new_doc_ids &= old_doc_ids

        for term in q_terms:
            term_clean_positions = {}
            term_positions = self.index.positional_postings(term)
            for doc_positions in term_positions:
                if doc_positions[0] in new_doc_ids:
                    term_clean_positions[doc_positions[0]] = doc_positions[1]
            final_positions[term] = term_clean_positions

        for doc_id in new_doc_ids:
            rank.push(self.index.doc_path(doc_id), 
                         self.score(final_positions, doc_id, q_terms))

        return rank

    def score(self, final_positions, doc_id, q_terms):
        pos_list = []
        q_len = len(q_terms)

        for term in q_terms: 
            pos_list.append(final_positions[term][doc_id] + [math.inf])

        if q_len == 1:
            return float(len(pos_list[0]) - 1)

        a = - 1
        score = 0
        pointer = [0] * q_len
        b = max([t_list[0] for t_list in pos_list])

        while b != math.inf:
            i = 0
            for j in range(q_len):
                while pos_list[j][pointer[j]+1] <= b:
                    pointer[j] += 1
                if pos_list[j][pointer[j]] < pos_list[i][pointer[i]]:
                    i = j
            a = pos_list[i][pointer[i]]
            score += 1 / (b - a - q_len + 2)
            b = pos_list[i][pointer[i]+1]

        return score


### Explicación/documentación

4.1 - ProximitySearcher

La implementación de este algoritmo ha presentado un par de problemas, nos hemos basado en ejercicios de teoria y posts de internet.

Comenzamos parseando la query y aplicando el cutoff para observar cuales son los documentos validos para la consulta.
Cogemos los documentos que contengan el primer termino y aplicamos la interseccion con los que contienen el segundo, la solucion la intersecamos con el tercer termino y asi sucesivamente con todos. Tendremos asi un set con los ids de los documentos que contienen todos los terminos de la consulta, una vez tenemos esto, limpiamos via positional postings. lo que hacemos es basicamente llamar a positional postings eliminando todos los documentos que no ficuren en el set anteriormente sacado, una vez tenemos estopara cada documento llamamos a la funcion score, que obtendrá B y A de cada grupo de elementos y calculará el score.

# Ejercicio 5: Índice posicional* (1pt)

Implementar una variante adicional de índice (como subclase si se considera oportuno) que extienda las estructuras de índices con la inclusión de posiciones en las listas de postings. La implementación incluirá una clase PositionalIndexBuilder para la construcción del índice posicional así como una clase PositionalIndex para proporcionar acceso al mismo.

In [ ]:
class PositionalIndex(RAMIndex):
    # Your new code here (exercise 5*) #
    # Note that it may be better to inherit from a different class
    # if your index extends a particular type of index
    # For example: PositionalIndex(RAMIndex)
    def __init__(self, dir_path=None):
        self.terms_dict = {}
        super().__init__(dir_path)
    
    def positional_postings(self, term):
        dict = self.terms_dict[term]
        returned_postings = [] 
        for doc, pos in dict.items():
            returned_postings.append(tuple((doc, pos))) 
        
        return returned_postings

    def postings(self, term):
        dict = self.terms_dict[term]
        returned_postings = []
        for doc, pos in dict.items():
            returned_postings.append((doc, len(pos)))
        return returned_postings


class PositionalIndexBuilder(RAMIndexBuilder):
    # Your new code here (exercise 5*) #
    # Same note as for PositionalIndex
    def __init__(self, dir_path, parser=BasicParser()):
        super().__init__(dir_path, parser)
        self.index = PositionalIndex(dir_path)

    def index_document(self, path, text):
        doc_id = self.index.ndocs()

        self.index.add_doc(path)

        text = BasicParser.parse(text)
        term_freq: Dict = {}

        for pos, term in enumerate(text):
            if self.index.terms_dict.get(term) is None:
                term_freq[term] = []
                self.index.terms_dict[term] = {}

            elif term_freq.get(term) is None:
                term_freq[term] = []

            term_freq[term].append(pos)

        for term in term_freq.keys():
            self.index.terms_dict[term][doc_id] = term_freq[term]

        return


### Explicación/documentación, indicando además el tipo de índice que se ha implementado y los aspectos que sean destacables

5.1 - PositionalIndex

Para la implementacion de este ejercicio, hemos hecho que se herede de la clase RAMIndex, y para la implementación simplemente se ha iterado sobre las claves del diccionario y comprobalo la longitud de los postings dando la frecuencia. La lista de postings posicionales se ha obtenido iterando sobre su diccionario y obteninedo la lista de posiciones de cada id de documetno, el resto sigue el mismo flujo que el RAMIndex

5.2 - PositionalIndexBuilder

Para construir el índice hemos almacenado posiciones, con lo que vamos a tener un índice que sea un diccionario término-diccionario, donde este segundo diccionario será uno con la forma docid-posiciones. Así, el diccionario auxiliar que teníamos antes con la forma término-frecuencia, se convierte en uno de la forma término-posiciones, con lo que podremos conocer las posiciones de manera sencilla.

# Ejercicio 6: PageRank (1pt)

Implementar el algoritmo PageRank en una clase PagerankDocScorer, que permitirá devolver un ranking de los documentos de manera similar a como hace un Searcher (pero sin recibir una consulta). 

Se recomienda, al menos inicialmente, llevar a cabo una implementación con la que los valores de PageRank sumen 1, para ayudar a la validación de la misma. Posteriormente, si se desea, se pueden escalar (o no, a criterio del estudiante) los cálculos omitiendo la división por el número total de páginas en el grafo. Será necesario tratar los nodos sumidero tal como se ha explicado en las clases de teoría.

In [ ]:
class PagerankDocScorer():
    def __init__(self, graphfile, r, n_iter):
        self.n_iter = n_iter

        f = open(graphfile, "r")
        lines = f.read().splitlines()
        f.close()

        self.nodos = set()  # Elementos sin ordenar y sin repetir #
        self.salientes: Dict = {}  # Clave: nodo, valor: lista de a quienes apunta #
        self.nodos_desde_hacia: Dict = {}  # Clave: nodo, valor: lista de quienes le apuntan #

        # Voy viendo donde apunta cada nodo y almacenando los nodos #
        for line in lines:
            l = line.split('\t')
            self.nodos.add(l[0])
            self.nodos.add(l[1])

            # Comprobar salidas de cada nodo #

            try:
                self.nodos_desde_hacia[l[1]].append(l[0])
            except:
                self.nodos_desde_hacia[l[1]] = [l[0]]

            try:
                self.salientes[l[0]].append(l[1])
            except:
                self.salientes[l[0]] = [l[1]]

            # Sumideros #

            try:
                if self.nodos_desde_hacia[l[0]]:
                    pass
            except:
                self.nodos_desde_hacia[l[0]] = []

            try:
                if self.salientes[l[1]]:
                    pass
            except:
                self.salientes[l[1]] = []


        # Guardamos los sumideros #
        self.sumideros = []
        for nodo in self.salientes.keys():
            if len(self.salientes[nodo]) == 0:
                self.sumideros.append(nodo)

        self.n = len(self.nodos)
        self.r = r

    def rank(self, cutoff):
        p1 = self.r / self.n
        p2 = 1 - self.r
        res = []
        pageRankActual = {}
        pageRankAnterior = {}
        

        # Inicializar valores (se omite la division entre N) #
        for n in self.nodos:
            pageRankAnterior[n] = 1

        # Computar el PageRank iterativamente #
        for aux in range(self.n_iter):
            for n in self.nodos:
                sum = 0
                # Se suma cada nodo que va hacia el que iteramos y luego se suman los sumideros #
                for origin in self.nodos_desde_hacia[n]:
                    sum += pageRankAnterior[origin] / len(self.salientes[origin])
                    
                for i in self.sumideros:
                    sum += pageRankAnterior[i] / self.n

                pageRankActual[n] = p1 + p2 * sum

            # Guardamos valores PageRank #
            for n in pageRankActual.keys():
                pageRankAnterior[n] = pageRankActual[n]

        # Ordenamos según PageRank #
        ordered = dict(sorted(pageRankActual.items(),
                              key=lambda item: item[1], reverse=True))
        aux = 0
        for k, v in ordered.items():
            res.append((k, v))
            aux += 1
            if aux == cutoff:
                break
        # Devolvemos una lista [(nodo, pageRankScore), (nodo, pageRankScore), etc] #
        return res

### Explicación/documentación

Para inicializar el PageRank vamos a leer de un archivo y vamos a establecer tres tipos de grupos: los nodos en general, las salidas que tiene un nodo (un nodo a que nodos va) y los nodos que entran en otro nodo (los nodos que llegan a un nodo concreto). Por cada línea del archivo vemos dónde apunta cada uno y contamos las salidas de cada nodo además de las entradas que recibe ese nodo. Finalmente, aquellos nodos que no tienen salidas son considerados sumideros, y por lo tanto los guardamos en una categoría aparte (self.sumideros).

Para hacer el rank, inicializamos todos los pageRanks a 1 para que todos los nodos estén igualados al principio. A continuación, por un lado, iteraremos por todos los nodos y sumaremos todos los nodos que van hacia el nodo en el que estamos y, por otro lado, sumaremos los sumideros. Una vez hemos iterado todos los nodos reordenamos en función del PageRank y devolvemos una lista del tamaño del cutoff en la viene el nodo y el PageRank del mismo.

# Programa de prueba **main**

Descarga los ficheros del curso de Moodle y coloca sus contenidos en una carpeta **collections** en el mismo directorio que este *notebook*. El fichero <u>toys.zip</u> hay que descomprimirlo para indexar las carpetas que contiene. Igualmente, el fichero <u>graphs.zip</u> incluye ficheros (*1k-links.dat*, *toy-graph1.dat*, *toy-graph2.dat*) que se deben descomprimir en la carpeta collections para que el main funcione.

## Función **main**

In [ ]:
import os
import shutil
import psutil
import time

def main():
    index_root_dir = "./index/"
    collections_root_dir = "./collections/"
    test_collection (collections_root_dir + "toys/toy1/", index_root_dir + "toys/toy1/", "cc", ["aa dd", "aa"], True)
    test_collection (collections_root_dir + "toys/toy2/", index_root_dir + "toys/toy2/", "aa", ["aa cc", "bb aa"], True)
    test_collection (collections_root_dir + "urls.txt", index_root_dir + "urls/", "wikipedia", ["information probability", "probability information", "higher probability"], False)
    test_collection (collections_root_dir + "docs1k.zip", index_root_dir + "docs1k/", "seat", ["obama family tree"], True)
    test_collection (collections_root_dir + "docs10k.zip", index_root_dir + "docs10k/", "seat", ["obama family tree"], True)
    test_pagerank("./collections/", 5)

def test_collection(collection_path: str, index_path: str, word: str, queries: list, analyse_performance: bool):
    print("=================================================================")
    print("Testing indices and search on " + collection_path)

    # # We now test building different implementations of an index
    test_build(WhooshBuilder(index_path + "whoosh"), collection_path)
    test_build(WhooshForwardBuilder(index_path + "whoosh_fwd"), collection_path)
    test_build(WhooshPositionalBuilder(index_path + "whoosh_pos"), collection_path)
    test_build(RAMIndexBuilder(index_path + "ram"), collection_path)
    test_build(DiskIndexBuilder(index_path + "disk"), collection_path)
    test_build(PositionalIndexBuilder(index_path + "pos"), collection_path)

    # # We now inspect all the implementations
    indices = [
            WhooshIndex(index_path + "whoosh"),
            WhooshForwardIndex(index_path + "whoosh_fwd"), 
            WhooshPositionalIndex(index_path + "whoosh_pos"), 
            RAMIndex(index_path + "ram"),
            DiskIndex(index_path + "disk"),
            PositionalIndex(index_path + "pos"),
            ]
    for index in indices:
        test_read(index, word)

    for query in queries:
        print("------------------------------")
        print("Checking search results for %s" % (query))
        # Whoosh searcher can only work with its own indices
        test_search(WhooshSearcher(index_path + "whoosh"), WhooshIndex(index_path + "whoosh"), query, 5)
        test_search(WhooshSearcher(index_path + "whoosh_fwd"), WhooshForwardIndex(index_path + "whoosh_fwd"), query, 5)
        test_search(WhooshSearcher(index_path + "whoosh_pos"), WhooshPositionalIndex(index_path + "whoosh_pos"), query, 5)
        test_search(ProximitySearcher(WhooshPositionalIndex(index_path + "whoosh_pos")), WhooshPositionalIndex(index_path + "whoosh_pos"), query, 5)
        for index in indices:
            # our searchers should work with any other index
            test_search(SlowVSMSearcher(index), index, query, 5)
            test_search(TermBasedVSMSearcher(index), index, query, 5)
            test_search(DocBasedVSMSearcher(index), index, query, 5)
        test_search(ProximitySearcher(PositionalIndex(index_path + "pos")), PositionalIndex(index_path + "pos"), query, 5)

    # if we keep the list in memory, there may be problems with accessing the same index twice
    indices = list()

    if analyse_performance:
        # let's analyse index performance
        test_index_performance(collection_path, index_path)
        # let's analyse search performance
        for query in queries:
            test_search_performance(collection_path, index_path, query, 5)

def test_build(builder, collection):
    stamp = time.time()
    print("Building index with", type(builder))
    print("Collection:", collection)
    # this function should index the recieved collection and add it to the index
    builder.build(collection)
    # when we commit, the information in the index becomes persistent
    # we can also save any extra information we may need
    # (and that cannot be computed until the entire collection is scanned/indexed)
    builder.commit()
    print("Done (", time.time() - stamp, "seconds )")
    print()

def test_read(index, word):
    stamp = time.time()
    print("Reading index with", type(index))
    print("Collection size:", index.ndocs())
    print("Vocabulary size:", len(index.all_terms()))
    # more tests
    doc_id = 0
    print("  Frequency of word \"" + word + "\" in document " + str(doc_id) + " - " + index.doc_path(doc_id) + ": " + str(index.term_freq(word, doc_id)))
    print("  Total frequency of word \"" + word + "\" in the collection: " + str(index.total_freq(word)) + " occurrences over " + str(index.doc_freq(word)) + " documents")
    print("  Docs containing the word '" + word + "':", index.doc_freq(word))
    print("    First two documents:", [(doc, freq) for doc, freq in index.postings(word)][0:2])
    print("Done (", time.time() - stamp, "seconds )")
    print()


def test_search (engine, index, query, cutoff):
    stamp = time.time()
    print("  " + engine.__class__.__name__ + " with index " + index.__class__.__name__ + " for query '" + query + "'")
    for path, score in engine.search(query, cutoff):
        print(score, "\t", path)
    print()
    print("Done (", time.time() - stamp, "seconds )")
    print()

def disk_space(index_path: str) -> int:
    space = 0
    for f in os.listdir(index_path):
      p = os.path.join(index_path, f)
      if os.path.isfile(p):
        space += os.path.getsize(p)
    return space


def test_index_performance (collection_path: str, base_index_path: str):

    pid = os.getpid()
    meminfo = psutil.Process(pid)
    
    print("----------------------------")
    print("Testing index performance on " + collection_path + " document collection")

    print("===============================================================")
    print("  Build time...")

    start_time = time.time()
    b = WhooshBuilder(base_index_path + "whoosh")
    b.build(collection_path)
    b.commit()
    print("\tWhooshIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    start_time = time.time()
    b = WhooshForwardBuilder(base_index_path + "whoosh_fwd")
    b.build(collection_path)
    b.commit()
    print("\tWhooshForwardIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    start_time = time.time()
    b = WhooshPositionalBuilder(base_index_path + "whoosh_pos")
    b.build(collection_path)
    b.commit()
    print("\tWhooshPositionalIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    start_time = time.time()
    b = RAMIndexBuilder(base_index_path + "ram")
    b.build(collection_path)
    b.commit()
    print("\tRAMIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    start_time = time.time()
    b = DiskIndexBuilder(base_index_path + "disk")
    b.build(collection_path)
    b.commit()
    print("\tDiskIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    start_time = time.time()
    b = PositionalIndexBuilder(base_index_path + "positional index")
    b.build(collection_path)
    b.commit()
    print("\tPositionalIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    print("===============================================================")
    print("  Load time...")
    start_time = time.time()
    WhooshIndex(base_index_path + "whoosh")
    print("\tWhooshIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    start_time = time.time()
    WhooshForwardIndex(base_index_path + "whoosh_fwd")
    print("\tWhooshForwardIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    start_time = time.time()
    WhooshPositionalIndex(base_index_path + "whoosh_pos")
    print("\tWhooshPositionalIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    start_time = time.time()
    RAMIndex(base_index_path + "ram")
    print("\tRAMIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    start_time = time.time()
    DiskIndex(base_index_path + "disk")
    print("\tDiskIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    start_time = time.time()
    PositionalIndex(base_index_path + "disk")
    print("\tPositionalIndex: %s seconds ---" % (time.time() - start_time))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    print("===============================================================")
    print("  Disk space...")
    print("\tWhooshIndex: %s space ---" % (disk_space(base_index_path + "whoosh")))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    print("\tWhooshForwardIndex: %s space ---" % (disk_space(base_index_path + "whoosh_fwd")))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    print("\tWhooshPositionalIndex: %s space ---" % (disk_space(base_index_path + "whoosh_pos")))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    print("\tRAMIndex: %s space ---" % (disk_space(base_index_path + "ram")))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    print("\tDiskIndex: %s space ---" % (disk_space(base_index_path + "disk")))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")

    print("\tPositionalIndex: %s space ---" % (disk_space(base_index_path + "positional index")))
    memoryUse = meminfo.memory_info()[0]/1024/1024
    print(f"\tMemory usage in MB: {memoryUse}\n")



def test_search_performance (collection_name: str, base_index_path: str, query: str, cutoff: int):
    print("----------------------------")
    print("Testing search performance on " + collection_name + " document collection with query: '" + query + "'")
    whoosh_index = WhooshIndex(base_index_path + "whoosh")
    ram_index = RAMIndex(base_index_path + "ram")
    disk_index = DiskIndex(base_index_path + "disk")
    positional_index = PositionalIndex(base_index_path + "positional index")

    start_time = time.time()
    test_search(WhooshSearcher(base_index_path + "whoosh"), whoosh_index, query, cutoff)
    print("--- Whoosh on Whoosh %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    test_search(SlowVSMSearcher(whoosh_index), whoosh_index, query, cutoff)
    print("--- SlowVSM on Whoosh %s seconds ---" % (time.time() - start_time))

    # # let's test some combinations of ranking + index implementations
    start_time = time.time()
    test_search(TermBasedVSMSearcher(whoosh_index), whoosh_index, query, cutoff)
    print("--- TermVSM on Whoosh %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    test_search(TermBasedVSMSearcher(ram_index), ram_index, query, cutoff)
    print("--- TermVSM on RAM %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    test_search(TermBasedVSMSearcher(disk_index), disk_index, query, cutoff)
    print("--- TermVSM on Disk %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    test_search(TermBasedVSMSearcher(positional_index), positional_index, query, cutoff)
    print("--- TermVSM on PositionalIndex %s seconds ---" % (time.time() - start_time))

    start_time = time.time()
    test_search(DocBasedVSMSearcher(ram_index), ram_index, query, cutoff)
    print("--- DocVSM on RAM %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    test_search(DocBasedVSMSearcher(disk_index), disk_index, query, cutoff)
    print("--- DocVSM on Disk %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    test_search(DocBasedVSMSearcher(positional_index), positional_index, query, cutoff)
    print("--- DocVSM on PositionalIndex %s seconds ---" % (time.time() - start_time))

def test_pagerank(graphs_root_dir, cutoff):
    print("----------------------------")
    print("Testing PageRank")
    # we separate this function because it cannot work with all the collections
    start_time = time.time()
    for path, score in PagerankDocScorer(graphs_root_dir + "toy-graph1.dat", 0.5, 50).rank(cutoff):
        print(score, "\t", path)
    print()
    print("--- Pagerank with toy_graph_1 %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    for path, score in PagerankDocScorer(graphs_root_dir + "toy-graph2.dat", 0.6, 50).rank(cutoff):
        print(score, "\t", path)
    print()
    print("--- Pagerank with toy_graph_2 %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    for path, score in PagerankDocScorer(graphs_root_dir + "1k-links.dat", 0.2, 50).rank(cutoff):
        print(score, "\t", path)
    print()
    print("--- Pagerank with simulated links for doc1k %s seconds ---" % (time.time() - start_time))

main()


### Resumen de coste y rendimiento

Para el consumo de RAM no hemos consiserado optimo utillizar un porcentaje debido a que lo hemos ejecutado en un equipo windows por lo que los porcenatajes pueden variar mucho sin contar con el programa ejecutado asique en lugar del porcentaje visto en cada momento hemos apuntado la cantidad de RAM utilizada por el proceso actual, esto lo hemos conseguuido haciendo uso de la libreria psutil, obteniendo el pid del proceso a la hora de hacer los tests y cogiendo los valores de la funcion memory_info() siendo el primer valor el rss del equipo.

Hemos optado por dejar los cambios en el main para comparacion de resultados.

La medida se ha hecho de forma individual, al ejecutar el ejercicio entero los resultados pueden variar, la entrega se ha hecho ejecutando uno a uno y reiniciando el notebook con cada ejecución

El equipo utilizado tiene las siguientes caracteristicas

    Total memoria RAM: 16 GB
    Procesador: AMD Ryzen 5 5500U with Radeon Graphics 2.10 GHz
    Numero de nucleos: 6 Procesadores fisicos (12 Procesadores lógicos)

 -

|  RAMIndex      | Construcción |del | índice | Carga |del  índice |
|------|--------------------|-----------------|------------------|-----------------|-----------------|
|      | Tiempo de indexado | Consumo máx RAM | Espacio en disco | Tiempo de carga | Consumo máx RAM |
| toy1 | 0.004004478454589844 | 80.12890625 MB | 1312 | 0.0009987354278564453 | 80.15234375 MB |
| toy2 | 0.004998445510864258 | 80.90234375 MB | 1199 | 0.0010004043579101562 | 80.90625 MB |
| 1K | 36.583067417144775 | 233.8515625 MB | 7264763 | 0.9582297801971436 | 340.30078125 MB |
| 10K | 260.15782046318054 | 807.95703125 MB | 51154568 | 15.77506160736084 | 1499.16796875 MB |

 -

|   DISKIndex      | Construcción |del | índice | Carga| del  índice |
|------|--------------------|-----------------|------------------|-----------------|-----------------|
|      | Tiempo de indexado | Consumo máx RAM | Espacio en disco | Tiempo de carga | Consumo máx RAM |
| toy1 | 0.05399942398071289 | 80.12890625 MB | 1266 | 0.0019991397857666016 | 80.15234375 MB |
| toy2 | 0.004998445510864258 | 80.90234375 MB | 1162 | 0.0019991397857666016 | 80.90625 MB |
| 1K | 60.96139931678772 | 239.1015625 MB | 5978805 | 0.04128122329711914 | 340.30078125 MB |
| 10K | 335.91592717170715 | 838.42578125 MB | 42905316 | 0.19899725914001465 | 1496.41796875 MB |

 -

|   PositionalIndex   | Construcción |del | índice | Carga |del  índice |
|------|--------------------|-----------------|------------------|-----------------|-----------------|
|      | Tiempo de indexado | Consumo máx RAM | Espacio en disco | Tiempo de carga | Consumo máx RAM |
| toy1 | 0.04098963737487793 | 80.12890625 MB | 13329 | 0.002002239227294922 | 80.15234375 MB |
| toy2 | 0.04300403594970703 | 80.90234375 MB | 12642 | 0.0019948482513427734 | 80.90625 MB |
| 1K | 38.02010130882263 | 333.89844375 MB | 11907362 | 0.047033071517944336 | 340.30078125 MB |
| 10K | 271.85889172554016 | 1489.2421875 MB | 79015017 | 0.201005220413208 | 1497.11328125 MB |
